# Install Detectron2

**Make Sure you run this notebook on GPU**

In [ ]:
# install pyyaml==5.1 to prevent dependency issue
!python -m pip install pyyaml==5.1

# install detectron2
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

**Check installation**

In [2]:
import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0
torch:  2.0 ; cuda:  cu118
detectron2: 0.6


# Get Dataset

I got the original data from here https://universe.roboflow.com/dissertation-bqltf/apple-detection-leqnn/dataset/2

I re-annotated the images using pre-trained mask-rcnn models and use the new annotated images for this training

**Install latest gdown library**

In [ ]:
!pip install -U gdown

**Download and Extract Data**

In [ ]:
!gdown https://drive.google.com/uc?id=1-HCo6R4_wI8iFhNsZrat4VlXkvJ7VSuW
!unzip '/content/apple_coco_dataset.zip'

# Training

**Import required libraries**

In [5]:
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow

from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor, DefaultTrainer
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

**Register Our Dataset**

In [6]:
from detectron2.data.datasets import register_coco_instances

register_coco_instances("apple_train", {}, "/content/train_annotations.json", "/content/train/")
register_coco_instances("apple_val", {}, "/content/valid_annotations.json", "/content/valid")

MetadataCatalog.get("apple_train").thing_classes = ["apple"]
MetadataCatalog.get("apple_val").thing_classes = ["apple"]

**Setup the config for the training**

I use Mask R-CNN 50 FPN 3x which has good balance in speed and accuracy

In [7]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("apple_train",)
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.0001
cfg.SOLVER.MAX_ITER = 300
cfg.SOLVER.STEPS = []
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1
cfg.MODEL.RETINANET.NUM_CLASSES = 1
cfg.MODEL.ROI_KEYPOINT_HEAD.NUM_KEYPOINTS = 1
cfg.MODEL.SEM_SEG_HEAD.NUM_CLASSES = 1
cfg.OUTPUT_DIR = "/content/apple_train_output"


In [ ]:
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

**make sure you download the model or move it to your own storage**